In [2]:
import wrds
import pandas as pd

In [3]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection(wrds_username='alexandregg')

Loading library list...
Done


In [5]:
sp500 = conn.raw_sql("""
                        select a.*
                        from crsp.dsp500 as a
                        """)
sp500

,caldt,vwretd,vwretx,ewretd,ewretx,totval,totcnt,usdval,usdcnt,spindx,sprtrn
0,1925-12-31,NaN,NaN,NaN,NaN,1.523683e+07,89.0,NaN,NaN,NaN,NaN
1,1926-01-02,0.004297,0.004297,0.002941,0.002941,1.531969e+07,89.0,1.523683e+07,79.0,NaN,NaN
2,1926-01-04,-0.001357,-0.001357,0.001036,0.001036,1.529890e+07,89.0,1.531969e+07,80.0,NaN,NaN
3,1926-01-05,-0.004603,-0.004653,-0.005856,-0.006063,1.522771e+07,89.0,1.529890e+07,80.0,NaN,NaN
4,1926-01-06,0.000537,0.000537,0.000888,0.000888,1.523589e+07,89.0,1.522771e+07,80.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
25355,2022-03-25,0.004894,0.004886,0.007028,0.007009,4.003997e+10,505.0,3.984528e+10,505.0,4543.06,0.005066
25356,2022-03-28,0.007666,0.007665,0.002347,0.002343,4.034687e+10,505.0,4.003997e+10,505.0,4575.52,0.007145
25357,2022-03-29,0.012267,0.012267,0.016270,0.016270,4.084181e+10,505.0,4.034687e+10,505.0,4631.60,0.012257
25358,2022-03-30,-0.006201,-0.006307,-0.007469,-0.007657,4.058423e+10,505.0,4.084181e+10,505.0,4602.45,-0.006294


In [6]:
sp500.to_csv("sp500_price.csv")
sp500 = sp500[["caldt", "vwretx", "vwretd"]]
sp500.columns = ['date', 'vwReturnNoDiv', 'vwReturnDiv']
sp500.to_parquet("spx_price.parquet")

In [7]:
ticker_list = sp500.ticker.unique()
with open("ticker_list.txt", "w") as f:
    for ticker in ticker_list:
        f.write(f"{ticker}\n")

In [13]:
sp500.columns

Index(['permno', 'start', 'ending', 'cusip', 'permno', 'permco', 'issuno',
       'hexcd', 'hsiccd', 'date', 'bidlo', 'askhi', 'prc', 'vol', 'ret', 'bid',
       'ask', 'shrout', 'cfacpr', 'cfacshr', 'altprc', 'spread', 'altprcdt',
       'retx'],
      dtype='object')

In [12]:
with open("delta_list.txt", "r") as f:
    delta_list = [line.replace("\n", "") for line in f]

delta_list = tuple(delta_list)
delta_list

('ACF',
 'ABK',
 'NCM',
 'AMT',
 'SUNW',
 'JAVA',
 'AN',
 'ORCL',
 'MSFT',
 'SNDT',
 'SDS',
 'AYP',
 'AYE',
 'AYP',
 'AYE',
 'TROW',
 'ACD',
 'ALD',
 'HON',
 'EMCS',
 'EMC',
 'AH',
 'AH',
 'AH',
 'AMX',
 'AF',
 'AHD',
 'AT',
 'AMB',
 'FO',
 'BEAM',
 'ABP',
 'ABC',
 'ABP',
 'ABC',
 'ABP',
 'ABC',
 'ABP',
 'ABC',
 'AC',
 'PA',
 'ACS',
 'ACJ',
 'LLTC',
 'AMO',
 'GENZ',
 'ADPT',
 'SXCL',
 'AR',
 'AST',
 'AST',
 'T',
 'ZA',
 'ASR',
 'ASR',
 'ASR',
 'AD',
 'A',
 'A',
 'ADM',
 'ADM',
 'AM',
 'DG',
 'PSVB',
 'SVRN',
 'SOV',
 'SF',
 'ASF',
 'AX',
 'AFI',
 'ARC',
 'FRBC',
 'FRC',
 'HSRC',
 'HRC',
 'HLS',
 'EHC',
 'FISV',
 'COFI',
 'CF',
 'BYK',
 'BLS',
 'BX',
 'BS',
 'SGIC',
 'SGI',
 'BG',
 'BG',
 'BU',
 'KSE',
 'BU',
 'KSE',
 'BWS',
 'BG',
 'CAL',
 'BC',
 'BGH',
 'UIS',
 'BGH',
 'UIS',
 'CERN',
 'CMVT',
 'CIT',
 'CFG',
 'CPC',
 'BFO',
 'CAH',
 'CP',
 'CP',
 'ADCO',
 'LDDSA',
 'LDDS',
 'WCOM',
 'JI',
 'DELL',
 'CDP',
 'DH',
 'CNN',
 'DH',
 'CNN',
 'CO',
 'CGG',
 'C',
 'AP',
 'CDL',
 'CITA',
 'AP

In [14]:
daily_data = conn.raw_sql(f"""
                        select a.*, b.ticker, b.comnam
                        from crsp.dsf as a,
                        crsp.msenames as b
                        where a.permno=b.permno
                        and a.date > '1999-01-01'
                        and ticker in {delta_list}
                        """)
daily_data

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,bid,ask,shrout,cfacpr,cfacshr,openprc,numtrd,retx,ticker,comnam
0,36720410,10001.0,7953.0,10398.0,2.0,4925.0,1999-01-04,9.500000,9.500000,9.500000,...,9.500000,9.750000,2421.0,1.5,1.5,9.500000,1.0,-0.012987,EGAS,ENERGY INC
1,36720410,10001.0,7953.0,10398.0,2.0,4925.0,1999-01-05,10.000000,10.000000,10.000000,...,9.500000,10.000000,2421.0,1.5,1.5,10.000000,3.0,0.052632,EGAS,ENERGY INC
2,36720410,10001.0,7953.0,10398.0,2.0,4925.0,1999-01-06,10.000000,10.000000,10.000000,...,9.750000,10.250000,2421.0,1.5,1.5,10.000000,8.0,0.000000,EGAS,ENERGY INC
3,36720410,10001.0,7953.0,10398.0,2.0,4925.0,1999-01-07,9.875000,10.250000,-10.062500,...,9.875000,10.250000,2421.0,1.5,1.5,NaN,0.0,0.006250,EGAS,ENERGY INC
4,36720410,10001.0,7953.0,10398.0,2.0,4925.0,1999-01-08,9.750000,9.750000,9.750000,...,9.750000,10.125000,2421.0,1.5,1.5,9.750000,2.0,-0.031056,EGAS,ENERGY INC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340541,88160R10,93436.0,53453.0,66252.0,3.0,9999.0,2022-03-25,997.320129,1021.799927,1010.640015,...,1010.210022,1010.890015,1033534.0,1.0,1.0,1008.000000,593068.0,-0.003235,TSLA,TESLA INC
340542,88160R10,93436.0,53453.0,66252.0,3.0,9999.0,2022-03-28,1053.599976,1097.879883,1091.839966,...,1091.839966,1092.199951,1033534.0,1.0,1.0,1065.099976,1113132.0,0.080345,TSLA,TESLA INC
340543,88160R10,93436.0,53453.0,66252.0,3.0,9999.0,2022-03-29,1073.109985,1114.770020,1099.569946,...,1099.819946,1099.959961,1033534.0,1.0,1.0,1107.989990,835619.0,0.007080,TSLA,TESLA INC
340544,88160R10,93436.0,53453.0,66252.0,3.0,9999.0,2022-03-30,1084.000000,1113.949951,1093.989990,...,1093.689941,1093.989990,1033534.0,1.0,1.0,1091.170044,598381.0,-0.005075,TSLA,TESLA INC


In [17]:
with open("ticker_list.txt", "r") as f:
    ticker_list = [line.replace("\n", "") for line in f]

In [21]:
ticker_list_master = daily_data.ticker.drop_duplicates().to_list()
for ticker in ticker_list:
    if ticker not in ticker_list_master:
        print(f"{ticker} not in list")

NCM not in list
ACJ not in list
BYK not in list
BU not in list
JI not in list
XA not in list
CUD not in list
MCAWA not in list
ESB not in list
EJN not in list
CUC not in list
CBO not in list
GUX not in list
GN not in list
GFO not in list
JWT not in list
KYR not in list
ND not in list
KSC not in list
LV not in list
LGT not in list
MSH not in list
MHT not in list
MTYRF not in list
MRW not in list
NKP not in list
NFK not in list
OT not in list
PFK not in list
PU not in list
RCA not in list
SJO not in list
SUO not in list
SX not in list
UV not in list
UNX not in list
VCA not in list
WMRY not in list
WIL not in list
WTH not in list
YAT not in list
YB not in list
ICR not in list
ABA not in list
PNDQ not in list
CSCS not in list
DN not in list
WSW not in list
BCW not in list
MLL not in list
CEH not in list
PET not in list
CMIL not in list
HLN not in list
ZB not in list
CLO not in list
LKB not in list
RVB not in list
RKS not in list
ZE not in list
SGIB not in list
CKL not in list
FIR not in li

In [22]:
daily_data.to_parquet("spx_stock_prices_master.parquet")